In [2]:
import fire
import os
import sys
import time
import json
import re
import random
import logging

import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
# from vllm import LLM, SamplingParams

from inference_utils import get_user_prompt, post_processing_answer
from embeddings_space import *

/space/hotel/phit/miniconda3/envs/zalo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Function to load the main model for text generation
def load_model(model_name, quantization, load_in):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        load_in_4bit=True if load_in == "4bit" else None,
        load_in_8bit=True if load_in == "8bit" else None,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    return model


# Function to load the PeftModel for performance optimization
def load_peft_model(model, peft_model):
    peft_model = PeftModel.from_pretrained(model, peft_model)
    return peft_model

def main(
    # model_path,
    model_name,
    peft_model: str = None,
    quantization: bool = False,
    load_in: str = "4bit",
    max_new_tokens=100,  # The maximum numbers of tokens to generate
    test_file: str = "data/math_test.json",
    seed: int = 42,  # seed value for reproducibility
    do_sample: bool = True,  # Whether or not to use sampling ; use greedy decoding otherwise.
    min_length: int = None,  # The minimum length of the sequence to be generated, input prompt + min_new_tokens
    use_cache: bool = True,  # [optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
    top_p: float = 1.0,  # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
    temperature: float = 1.0,  # [optional] The value used to modulate the next token probabilities.
    top_k: int = 50,  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
    repetition_penalty: float = 1.0,  # The parameter for repetition penalty. 1.0 means no penalty.
    length_penalty: int = 1,  # [optional] Exponential penalty to the length that is used with beam-based generation.
    enable_azure_content_safety: bool = False,  # Enable safety check with Azure content safety api
    enable_sensitive_topics: bool = False,  # Enable check for sensitive topics using AuditNLG APIs
    enable_salesforce_content_safety: bool = True,  # Enable safety check with Salesforce safety flan t5
    max_padding_length: int = None,  # the max padding length to be used with tokenizer padding the prompts.
    use_fast_kernels: bool = False,  # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels
    log_filename: str = "log.txt",
    output_filepath: str = "jupyter_submission.csv",
    time_output_filepath: str = "time_submission.csv",
    **kwargs,
):
    logging.basicConfig(
        filename=log_filename,
        filemode='a',
        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
        datefmt='%H:%M:%S',
        level=logging.DEBUG
    )

    log = logging.getLogger(__name__)
    with open(test_file) as f:
        data = json.load(f)["data"]

    # Set the seeds for reproducibility
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)

    # model = LLM(model=model_path, seed=seed)

    model = load_model(model_name, quantization, load_in)
    if peft_model:
        model = load_peft_model(model, peft_model)

    model.eval()
    # sampling_params = SamplingParams(n=1, best_of=1, temperature=temperature, top_p=top_p, stop_token_ids=[2], max_tokens=max_new_tokens)

    tokenizer = AutoTokenizer.from_pretrained(peft_model)
    if tokenizer.eos_token_id is None:
        tokenizer.eos_token = "</s>"  # eos token is required for SFT
        log.info(f"Add eos token: {tokenizer.eos_token}")
    if tokenizer.pad_token_id is None:
        if tokenizer.unk_token_id is not None:
            tokenizer.pad_token = tokenizer.unk_token
        else:
            tokenizer.pad_token = tokenizer.eos_token
        log.info(f"Add pad token: {tokenizer.pad_token}")
    tokenizer.padding_side = "right"
    
    if use_fast_kernels:
        """
        Setting 'use_fast_kernels' will enable
        using of Flash Attention or Xformer memory-efficient kernels
        based on the hardware being used. This would speed up inference when used for batched inputs.
        """
        try:
            from optimum.bettertransformer import BetterTransformer

            model = BetterTransformer.transform(model)
        except ImportError:
            log.error(
                "Module 'optimum' not found. Please install 'optimum' it before proceeding."
            )
            results = []
    tokenizer_embedings, model_embedings = get_model_and_tokenizer()
    model_embedings.cuda()

    db = process_data(read_data())
    db_texts = db["information"].values.tolist()[:500]
    db["raw_texts"] = (
        "### Question:"
        + db["question"]
        + "### Choices: "
        + db["choices"]
        + "### Explanation: "
        + db["explanation"]
    )
    db_raw_texts = db["raw_texts"].values.tolist()[:500]
    log.info("Embedding database")
    import gc
    gc.collect()
    db_embeddings = embedding_text(
        tokenizer=tokenizer_embedings, model=model_embedings, input_texts=db_texts
    )
    print(db_embeddings.shape)
    # return None
    import gc
    gc.collect()


    # Dummy
    # model.generate(get_user_prompt(data[0]), n=1, best_of=1, temperature=temperature, top_p=top_p, stop_token_ids=[2], max_tokens=max_new_tokens)

    results = []
    times = []

    for idx, example in enumerate(data):
        log.info(f"Processing {idx}")
        start = time.time()
        relevant_examples = get_relevance_embeddings(
            db_embeddings,
            embedding_query_text(
                tokenizer=tokenizer_embedings,
                model=model_embedings,
                query_text=example["question"],
            ),
        )
        log.info("get relevant_question")
        relevant_examples = get_relevance_texts(
            input_texts=db_raw_texts, scores=relevant_examples, top_k=1
        )
        user_prompt = get_user_prompt(
            example, relevant_examples="\n".join(relevant_examples)
        )
        # user_prompt = get_user_prompt(example)
        id = example["id"]
        choices = example["choices"]

        # output = model.generate(user_prompt)[0]

        # prompt = output.prompt
        # gen_text = output.outputs[0].text
        
        input = tokenizer(user_prompt, return_tensors="pt")
        batch = {k: v.to("cuda") for k, v in input.items()}
        start = time.perf_counter()
        with torch.no_grad():
            outputs = model.generate(
                **batch,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                temperature=temperature,
                min_length=min_length,
                use_cache=use_cache,
                top_k=top_k,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                **kwargs,
            )

        prompt = tokenizer.decode(outputs[0], skip_special_tokens=True)
        gen_text = tokenizer.decode(
            outputs[0][input["input_ids"].shape[1] :], skip_special_tokens=True
        )

        print(f"Prompt: {prompt!r}, Generated text: {gen_text!r}")

        answer_text = None

        for text in gen_text.split("###"):
            if 'Final choice' in text:
                answer_text = text
                break

        if answer_text is None:
            answer_text = gen_text

        log.info(f"Output text: {prompt + gen_text}")
        log.info(f"Gen text {gen_text}")
        log.info(f"Answer text {answer_text}")

        answer = post_processing_answer(answer_text, choices)
        e2e_inference_time = int((time.time() - start) * 1000)
        log.info(f'Inference time: {e2e_inference_time}')
        log.info(f"Answer {answer}")
        if answer is None:
            answer = random.choice(choices)
            log.info(f"Random Answer {answer}")
        results.append({"id": id, "answer": answer})
        times.append({"id": id, "time": e2e_inference_time})

    result_df = pd.DataFrame.from_dict(results)
    result_df.to_csv(output_filepath, index=False)

    time_df = pd.DataFrame.from_dict(times)
    time_df.to_csv(time_output_filepath, index=False)

In [7]:
main(
    model_name="outputs-pt-zephyr-beta-v1/",
    peft_model="outputs-sft-zephyr-beta-v1/checkpoint-1000/",
    max_new_tokens=1024,
    temperature=0.1,
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]


(500, 768)


/space/hotel/phit/miniconda3/envs/zalo/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/space/hotel/phit/miniconda3/envs/zalo/lib/python3.10/site-packages/bitsandbytes-0.41.2-py3.10.egg/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:Bạn Nam mang hai tờ tiền có mệnh giá 10 000 đồng đi mua bút chì. Bạn mua hết 15 000. Bạn Nam còn thừa ...………đồng.### Choices: ['A. 10 000 đồng', 'B. 5 000 đồng', 'C. 2 000 đồng', 'D. 1 000 đồng']### Explanation: Đầu tiên, chúng ta cần hiểu rõ về câu hỏi. Bạn Nam mang theo hai tờ tiền mỗi tờ có mệnh giá 10 000 đồng, tức là tổng cộng bạn Nam có 10 000 đồng x 2 = 20 000 đồng.\n\nSau đó, bạn Nam đã dùng 15 000 đồng để mua bút chì. Vậy, số tiền bạn Nam đã dùng là 15 000 đồng.\n\nCuối cùng, để tìm ra số tiền bạn Nam còn lại sau khi mua bút chì, chúng ta lấy tổng số tiền bạn Nam có ban đầu trừ đi số tiền bạn Nam đã dùng để mua bút chì. Tức là: 20 000 đồng - 15 000 đồng = 5 000 đồng.\n\nVậy, bạn Nam còn thừa 5 000 đồng sau khi mua bút chì. Đáp án chính xác là 5 000 đồng.\n### Questi

In [6]:
main(
    model_name="outputs-pt-zephyr-beta-v1/",
    peft_model="outputs-sft-zephyr-beta-v1/checkpoint-1000/",
    load_in="8bit",
    max_new_tokens=1024,
    temperature=0.05,
    do_sample=False,
    output_filepath="jupyter_submission_new.csv",
    time_output_filepath="time_submission_new.csv",
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]


(500, 768)


/space/hotel/phit/miniconda3/envs/zalo/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/space/hotel/phit/miniconda3/envs/zalo/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Prompt: "Below is a math exercise. Provide a solution to that problem, if given multiple choices to answer; please give a final choice for solving that problem.\nSimilar Question and their explanation: ### Question:Bạn Nam mang hai tờ tiền có mệnh giá 10 000 đồng đi mua bút chì. Bạn mua hết 15 000. Bạn Nam còn thừa ...………đồng.### Choices: ['A. 10 000 đồng', 'B. 5 000 đồng', 'C. 2 000 đồng', 'D. 1 000 đồng']### Explanation: Đầu tiên, chúng ta cần hiểu rõ về câu hỏi. Bạn Nam mang theo hai tờ tiền mỗi tờ có mệnh giá 10 000 đồng, tức là tổng cộng bạn Nam có 10 000 đồng x 2 = 20 000 đồng.\n\nSau đó, bạn Nam đã dùng 15 000 đồng để mua bút chì. Vậy, số tiền bạn Nam đã dùng là 15 000 đồng.\n\nCuối cùng, để tìm ra số tiền bạn Nam còn lại sau khi mua bút chì, chúng ta lấy tổng số tiền bạn Nam có ban đầu trừ đi số tiền bạn Nam đã dùng để mua bút chì. Tức là: 20 000 đồng - 15 000 đồng = 5 000 đồng.\n\nVậy, bạn Nam còn thừa 5 000 đồng sau khi mua bút chì. Đáp án chính xác là 5 000 đồng.\n### Questi